In [1]:
import data_preprocess

data = data_preprocess.run(file_path = './ATP_rmsim.json',slidingwindow=False)


sequences = []
labels = []

MAX_LEN = 128

for item in data:
    while len(item['sequence']) > MAX_LEN:
        sequences.append(item['sequence'][:MAX_LEN])
        labels.append(item['label'][:MAX_LEN])
        item['sequence'] = item['sequence'][MAX_LEN:]
        item['label'] = item['label'][MAX_LEN:]
    sequences.append(item['sequence'])
    labels.append(item['label'])

In [2]:
import pandas as pd

zero_counter = 0
ones_counter = 0

for l in labels:
    zero_counter+=l.count(0)
    ones_counter+=l.count(1)

pd.DataFrame(columns=['sample amount'], data = [zero_counter, ones_counter])

,sample amount
0,542522
1,6250


In [3]:
from dataset import FADBindingDataset, split_dataset
from model import LCPLMforSequenceLabeling
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
model = LCPLMforSequenceLabeling("../../LCPLM/")
dataset = FADBindingDataset(tokenizer, sequences, labels, max_length=MAX_LEN)

/home/lovem/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at ../../LCPLM/ were not used when initializing LcPlmForMaskedLM: ['bimamba.backbone.layers.10.mixer.mamba_fwd.A_log', 'bimamba.backbone.layers.10.mixer.mamba_fwd.D', 'bimamba.backbone.layers.10.mixer.mamba_fwd.conv1d.bias', 'bimamba.backbone.layers.10.mixer.mamba_fwd.conv1d.weight', 'bimamba.backbone.layers.10.mixer.mamba_fwd.dt_proj.bias', 'bimamba.backbone.layers.10.mixer.mamba_fwd.dt_proj.weight', 'bimamba.backbone.layers.10.mixer.mamba_fwd.in_proj.weight', 'bimamba.backbone.layers.10.mixer.mamba_fwd.out_proj.weight', 'bimamba.backbone.layers.10.mixer.mamba_fwd.x_proj.weight', 'bimamba.backbone.layers.10.mixer.mamba_rev.A_log', 'bimamba.backbone.layers.10.mixer.mamba_rev.D', 'bimamba.backbone.l

In [4]:
dataset[0]

{'input_ids': tensor([ 0, 20,  7,  7,  4, 10, 16,  4, 10,  4,  4,  4, 22, 15, 17, 19, 11,  4,
         15, 15, 10, 15,  7,  4,  7, 11,  7,  4,  9,  4, 18,  4, 14,  4,  4, 18,
          8,  6, 12,  4, 12, 22,  4, 10,  4, 15, 12, 16,  8,  9, 17,  7, 14, 17,
          5, 11,  7, 19, 14, 13, 16, 21, 12, 16,  9,  4, 14,  4, 18, 18,  8, 18,
         14, 14, 14,  6,  6,  8, 22,  9,  4,  5, 19,  7, 14,  8, 21,  8, 13,  5,
          5, 10, 11, 12, 11,  9,  5,  7, 10, 10,  9, 18, 20, 12, 15, 20, 10,  7,
         21,  6, 18,  8,  8,  9, 15, 13, 18,  9, 13, 19,  7, 10, 19, 13, 17, 21,
          8,  2]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [5]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from transformers import get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter

[train_dataset, test_dataset] = split_dataset(dataset, 0.8)
[validate_dataset, test_dataset] = split_dataset(test_dataset, 0.5)


In [6]:
import numpy as np
from sklearn.metrics import classification_report, roc_curve

def find_best_evalution(logits, labels):
    logits_np = np.array(logits)
    labels_np = np.array(labels)
    #sigmoid probability
    print(logits_np.shape)
    probs = 1 / (1 + np.exp(-logits_np[:, :, 1]))#[batch size, sequence length]

    labels_flat = labels_np.flatten()
    probs_flat = probs.flatten()

     # 只取非 padding (-100) 的有效樣本
    active_indices = labels_flat != -100
    final_labels = labels_flat[active_indices]
    final_probs = probs_flat[active_indices]

    # 計算 ROC 並找出最佳 G-mean 閾值
    fpr, tpr, thresholds = roc_curve(final_labels, final_probs)
    gmeans = np.sqrt(tpr * (1 - fpr))
    ix = np.argmax(gmeans)
    best_threshold = thresholds[ix]

    print(f'Best Threshold: {best_threshold:.3f}, G-Mean: {gmeans[ix]:.3f}')
    return best_threshold
    

def custom_classification_report(logits, labels, test_evalution = False):
    """Filt the ignore value and return classification report"""
    labels_np = np.array(labels)
    logits_np = np.array(logits)
    #preds = np.argmax(logits_np, axis=-1).flatten()
    if test_evalution:
        threshold = find_best_evalution(logits, labels)
    else:
        threshold = 0.5
    probs = 1/(1+np.exp(-logits_np[:,:,1]))
    preds = (probs > threshold).astype(int).flatten()
    labels_flat = labels_np.flatten()

    print("Shape of flattened labels:", labels_flat.shape)
    print("Shape of flattened predictions:", preds.shape)

    active_indices = labels_flat != -100
    final_labels = labels_flat[active_indices]
    final_preds = preds[active_indices]
    report_str = classification_report(final_labels, final_preds)
    report_dict = classification_report(final_labels, final_preds, output_dict=True)
    print(report_str)
    return report_dict

In [7]:
from tqdm import tqdm
import yaml

def train_by_dataloader():

    writer = SummaryWriter('runs/my_experiment') # TensorBoard writer

    ## hyperparameters in yaml file
    with open("../config/mamba_config.yaml", "r") as f:
        config = yaml.safe_load(f)
    learning_rate = config["training_arguments"]["learning_rate"]
    per_device_train_batch_size = config["training_arguments"]["per_device_train_batch_size"]
    num_epochs = config["training_arguments"]["num_train_epochs"]
    warmup_steps = config['training_arguments']['warmup_steps']
    log_step = config['training_arguments']['eval_steps']
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    ##

    ## create dataloader
    train_loader = DataLoader(train_dataset, per_device_train_batch_size, shuffle=True)
    validation_loader = DataLoader(validate_dataset, per_device_train_batch_size, shuffle=True)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
    ## 

    ## some variable to use
    patience = 3
    best_val_loss = float('inf')
    best_f1_score = -float('inf')
    epochs_no_improve = 0
    best_model_state_dict = None
    global_step = 0
    ## 

    ## move model to GPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    ##
  
    for epoch in range(num_epochs):
        """Training Loop Start here"""
        model.train()  
        total_train_loss = 0  
            
        for step, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):          
            inputs = {k:v.to(device) for k,v in batch.items() if k != 'labels'}# 將批次數據移至設備  
            labels = batch['labels'].to(device)

            optimizer.zero_grad()# 清除梯度
            loss, _ = model(**inputs, labels=labels)  # 前向傳播  
            loss.backward()  # 反向傳播和優化  
            optimizer.step()
            scheduler.step()
            global_step+=1

            if global_step % log_step == 0:
                avg_train_loss = total_train_loss / (step + 1)
                writer.add_scalar('Loss/train', avg_train_loss, global_step)
                print(f"Epoch {epoch+1}, Step {step+1}, Training Loss: {avg_train_loss:.6f}")
                
            total_train_loss += loss.item()
        """Training Loop End here"""

        """Validate Loop Start here"""
        model.eval()
        total_eval_loss = 0
        all_val_logits = []
        all_val_labels = []
        with torch.no_grad():
            for batch in tqdm(validation_loader, desc=f"Validation Epoch {epoch+1}"):
                inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
                labels = batch['labels'].to(device)

                loss, logits = model(**inputs, labels=labels)

                total_eval_loss += loss.item()
                all_val_logits.extend(logits.cpu().numpy())
                all_val_labels.extend(labels.cpu().numpy())
        """Validate Loop End here"""

        """Report and Save the weight of the model"""
        report = custom_classification_report(all_val_logits, all_val_labels, True)

        avg_eval_loss = total_eval_loss / len(validation_loader)
        val_f1_score = report['1']['f1-score']##means Marco-f1-score on 1's sample prediction
        print(f"Validation Loss: {avg_eval_loss:.6f}")

        # Store the model has best performance on the specific indicator like: 
        # evaluation loss, recall rate or f1-score
        ## early stopping part
        if avg_eval_loss < best_val_loss:
            best_val_loss = avg_eval_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs!")
                break

        ## save weight part
        if val_f1_score > best_f1_score:
            best_f1_score = val_f1_score
            best_model_state_dict = model.state_dict() # 保存最佳權重
        """An epoch end here"""

        writer.add_scalar('Loss/val', avg_eval_loss, epoch + 1)
    writer.close()

    if best_model_state_dict is not None:
        model.load_state_dict(best_model_state_dict)
        print("Loaded best model weights!")
    else:
        print("No improvement found during training.")
    # 保存微調後的模型  
    torch.save(model.state_dict(), "atp_binding_model.pt")

In [8]:
import sys
sys.path.append('..')
from utils import helpers
import time

start_time = time.time()
helpers.log("Start Training")
trainer = train_by_dataloader()
helpers.log("End Training")
end_time = time.time()
helpers.log(f"Total Training Time:{end_time - start_time} seconds")

[2025-05-17 19:37:08]Start Training


Epoch 1:   0%|          | 0/919 [00:00<?, ?it/s]/home/lovem/miniconda3/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
Epoch 1:   0%|          | 1/919 [00:07<1:59:16,  7.80s/it]/home/lovem/miniconda3/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
Epoch 1:   4%|▎         | 33/919 [00:16<03:44,  3.95it/s] 

Epoch 1, Step 40, Training Loss: 0.166773


Epoch 1:   9%|▊         | 79/919 [00:24<03:35,  3.91it/s]

Epoch 1, Step 80, Training Loss: 0.141021


...

Epoch 8, Step 847, Training Loss: 0.000007


Epoch 8:  96%|█████████▋| 886/919 [03:50<00:09,  3.45it/s]

Epoch 8, Step 887, Training Loss: 0.000006


Validation Epoch 8: 100%|██████████| 115/115 [00:07<00:00, 15.51it/s]


(459, 128, 2)
Best Threshold: 0.090, G-Mean: 0.939
Shape of flattened labels: (58752,)
Shape of flattened predictions: (58752,)
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     53478
           1       0.29      0.91      0.44       689

    accuracy                           0.97     54167
   macro avg       0.64      0.94      0.71     54167
weighted avg       0.99      0.97      0.98     54167

Validation Loss: 0.000069
Early stopping triggered after 8 epochs!
Loaded best model weights!
[2025-05-17 20:09:17]End Training
[2025-05-17 20:09:17]Total Training Time:1928.83962059021 seconds


In [10]:
test_loader = DataLoader(test_dataset, batch_size = 4, shuffle=False)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.eval()
total_eval_loss = 0
all_val_logits = []
all_val_labels = []
with torch.no_grad():
    for batch in tqdm(test_loader):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        _, logits = model(**inputs)
        all_val_logits.extend(logits.cpu().numpy())
        all_val_labels.extend(labels.cpu().numpy())

custom_classification_report(all_val_logits, all_val_labels, True)
torch.cuda.empty_cache()

100%|██████████| 115/115 [00:03<00:00, 32.21it/s]

(460, 128, 2)
Best Threshold: 0.114, G-Mean: 0.951
Shape of flattened labels: (58880,)
Shape of flattened predictions: (58880,)
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     54962
           1       0.43      0.92      0.58       650

    accuracy                           0.98     55612
   macro avg       0.71      0.95      0.79     55612
weighted avg       0.99      0.98      0.99     55612

